In [3]:
from lxml import html
import requests
from collections import OrderedDict
import json
import argparse
import pandas as pd
import csv

def parse(input_file):
    restaurants = []
    for line in input_file:
        url = line
        response = requests.get(url).text
        parser = html.fromstring(response)
	
        XPATH_RATING = '//div[@id="ratingFilter"]//ul//li'

        XPATH_TRAVELER = '//div[@class="col segment extraWidth"]//ul//li'
        XPATH_TIME = '//div[@class="col season extraWidth"]//ul/li'
        XPATH_LANGUAGE = '//div[@class="col language extraWidth"]//ul/li'
#        XPATH_DETAILS = '//div[contains(@class,"details_tab")]//div[contains(@class, "table_section")]//text()'
        XPATH_NAME = '//h1[@property="name"]//text()'
        XPATH_HOTEL_RATING = '//span[@property="ratingValue"]//@content'
        XPATH_REVIEWS = '//a[@property="reviewCount"]/@content'
        XPATH_RANK = '//div[@class="slim_ranking"]//text()'
#         XPATH_OFFICIAL_DESCRIPTION = '//div[contains(@class,"additional_info")]//span[contains(@class,"tabs_descriptive_text")]//text()'
#         XPATH_OFFICIAL_DESCRIPTION = '//div[@class="additional_info"]//div'
        XPATH_LATITUDE = '//div[@class="mapContainer"]//@data-lat'
        XPATH_LONGITUDE = '//div[@class="mapContainer"]//@data-lng'
                                        
        ratings = parser.xpath(XPATH_RATING)
        travelers = parser.xpath(XPATH_TRAVELER)
        times = parser.xpath(XPATH_TIME)
        languages = parser.xpath(XPATH_LANGUAGE)
	
        raw_name = parser.xpath(XPATH_NAME)
        raw_rank = parser.xpath(XPATH_RANK)
        raw_review_count = parser.xpath(XPATH_REVIEWS)
        raw_rating = parser.xpath(XPATH_HOTEL_RATING)
#         raw_official_description = parser.xpath(XPATH_OFFICIAL_DESCRIPTION)
#        raw_details = parser.xpath(XPATH_DETAILS)
        raw_latitude = parser.xpath(XPATH_LATITUDE)
        raw_longitude = parser.xpath(XPATH_LONGITUDE)
					
        name = ''.join(raw_name).strip() if raw_name else None
        rank = ''.join(raw_rank).strip() if raw_rank else None
        review_count = ''.join(raw_review_count).strip() if raw_review_count else None
        hotel_rating = ''.join(raw_rating).strip() if raw_rating else None
#         official_description = ' '.join(' '.join(raw_official_description).split()) if raw_official_description else None
#        details = ' '.join(' '.join(raw_details).split()) if raw_details else None
        latitude = ' '.join(' '.join(raw_latitude).split()) if raw_latitude else None  
        longitude = ' '.join(' '.join(raw_longitude).split()) if raw_longitude else None 

        ratings_dict = OrderedDict()
        for rating in ratings:
            XPATH_RATING_KEY = './/div[@class="row_label"]//text()'
            XPATH_RATING_VALUE = './/span[@class="row_bar"]/following-sibling::span//text()'
            raw_rating_key = rating.xpath(XPATH_RATING_KEY)
            raw_rating_value = rating.xpath(XPATH_RATING_VALUE)
            cleaned_rating_key = ''.join(raw_rating_key).replace('\n','')
            cleaned_rating_value = ''.join(raw_rating_value).replace('\n','')
            ratings_dict.update({cleaned_rating_key:cleaned_rating_value})
    
        travelers_list = [] 
        for traveler in travelers:
            XPATH_TRAVELER_TYPE = './/label//text()'
            XPATH_TRAVELER_COUNT = './/span//text()'
            raw_traveler_type = traveler.xpath(XPATH_TRAVELER_TYPE)
            raw_traveler_count = traveler.xpath(XPATH_TRAVELER_COUNT)
            cleaned_traveler_type = ''.join(raw_traveler_type).replace('\n','').split(' ')
            cleaned_traveler_type = cleaned_traveler_type[0]
            cleaned_traveler_count = ''.join(raw_traveler_count).replace('\n','')
            travelers_list.append((cleaned_traveler_type,cleaned_traveler_count))
        
        times_list = []
        for time in times:
            XPATH_TIME_TYPE = './/label//text()'
            XPATH_TIME_COUNT = './/span//text()'
            raw_time_type = time.xpath(XPATH_TIME_TYPE)
            raw_time_count = time.xpath(XPATH_TIME_COUNT)
            cleaned_time_type = ''.join(raw_time_type).replace('\n','').split(' ')
            cleaned_time_type = cleaned_time_type[0]
            cleaned_time_count = ''.join(raw_time_count).replace('\n','')
            times_list.append((cleaned_time_type, cleaned_time_count))
        
        language_list = []
        for language in languages:
            XPATH_LANGUAGE_TYPE = './/label//text()'
            XPATH_LANGUAGE_COUNT = './/span//text()'
            raw_language_type = language.xpath(XPATH_LANGUAGE_TYPE)
            raw_language_count = language.xpath(XPATH_LANGUAGE_COUNT)
            cleaned_language_type = ''.join(raw_language_type).replace('\n','').split(' ')
            cleaned_language_type = cleaned_language_type[0]
            cleaned_language_count = ''.join(raw_language_count).replace('\n','')
            language_list.append(cleaned_language_type)
        language_list = language_list[1:]


        data = {'name':name,
				'rank':rank,
				'rating':hotel_rating,
                'review_count':review_count,
                'latitude':latitude,
                'longitude':longitude,
#                 'official_description':official_description,
                  'travelers':travelers_list,
                  'times of year':times_list,
                  'languages':language_list
#                  'details':details
	    }
        restaurants.append(data)
        
        
#    columns = ['property_title', 'rank', 'rating', 'review_count', 'location', 'official_description', 'travelers', 'times', 'langauges']
    df = pd.DataFrame(restaurants)
    return df



input_file = open('restaurants.txt', 'r')
scraped_data = parse(input_file)
scraped_data

,languages,latitude,longitude,name,rank,rating,review_count,times of year,travelers
0,"[English(88), German(3), Italian(3), ]",31.949797,35.925457,Turtle Green Tea Bar,#3 of 30 Coffee & Tea in Amman \n#59 of 595 Pl...,4.5,103,"[(Mar-May, (31)), (Jun-Aug, (24)), (Sep-Nov, (...","[(Families, (6)), (Couples, (9)), (Solo, (16))..."
1,"[English(10), French(1)]",31.94913,35.9292,Mr. Lollies,#3 of 25 Dessert in Amman \n#98 of 595 Places ...,5,11,"[(Mar-May, (3)), (Jun-Aug, (2)), (Sep-Nov, (5)...","[(Families, (1)), (Couples, (1)), (Solo, (0)),..."
2,[English(2)],None,None,Al-Raqaq Saj,#230 of 545 Restaurants in Amman,5,2,"[(Mar-May, (1)), (Jun-Aug, (1)), (Sep-Nov, (0)...","[(Families, (2)), (Couples, (0)), (Solo, (0)),..."
3,"[English(45), Arabic(5), Spanish(3), ]",31.94913,35.9292,Mijana Restaurant,#41 of 545 Restaurants in Amman,4.5,56,"[(Mar-May, (12)), (Jun-Aug, (15)), (Sep-Nov, (...","[(Families, (5)), (Couples, (10)), (Solo, (1))..."
4,"[English(43), Italian(5), French(2), ]",31.95133,35.93271,Zorba Touristic Restaurant,#149 of 545 Restaurants in Amman,4,53,"[(Mar-May, (11)), (Jun-Aug, (16)), (Sep-Nov, (...","[(Families, (8)), (Couples, (4)), (Solo, (1)),..."
5,"[English(49), Portuguese(2), Arabic(1), ]",31.94932,35.924484,Caffe Strada,#35 of 545 Restaurants in Amman,4.5,53,"[(Mar-May, (9)), (Jun-Aug, (9)), (Sep-Nov, (11...","[(Families, (2)), (Couples, (9)), (Solo, (11))..."
6,"[English(74), Arabic(3), Spanish(2)]",31.950348,35.923878,Firefly Burger,#70 of 545 Restaurants in Amman,4,79,"[(Mar-May, (21)), (Jun-Aug, (18)), (Sep-Nov, (...","[(Families, (13)), (Couples, (10)), (Solo, (4)..."
7,"[English(60), Arabic(3), German(1), ]",31.973589,35.86889,Tche Tche,#376 of 545 Restaurants in Amman,3,67,"[(Mar-May, (15)), (Jun-Aug, (18)), (Sep-Nov, (...","[(Families, (11)), (Couples, (5)), (Solo, (5))..."
8,[English(3)],31.95063,35.92315,Mr. Krepe,#463 of 545 Restaurants in Amman,2.5,3,"[(Mar-May, (0)), (Jun-Aug, (0)), (Sep-Nov, (1)...","[(Families, (0)), (Couples, (1)), (Solo, (0)),..."
9,"[English(170), Italian(7), French(5), ]",31.94937,35.92642,Al-Quds,#12 of 545 Restaurants in Amman,4.5,197,"[(Mar-May, (47)), (Jun-Aug, (38)), (Sep-Nov, (...","[(Families, (30)), (Couples, (33)), (Solo, (22..."


In [4]:
with open('restaurants_details2.csv', 'w') as csvfile:
    scraped_data.to_csv(csvfile,columns = ['latitude','longitude','name'], encoding='utf-8')

In [5]:
scraped_data.dropna()

,languages,latitude,longitude,name,rank,rating,review_count,times of year,travelers
0,"[English(88), German(3), Italian(3), ]",31.949797,35.925457,Turtle Green Tea Bar,#3 of 30 Coffee & Tea in Amman \n#59 of 595 Pl...,4.5,103,"[(Mar-May, (31)), (Jun-Aug, (24)), (Sep-Nov, (...","[(Families, (6)), (Couples, (9)), (Solo, (16))..."
1,"[English(10), French(1)]",31.94913,35.9292,Mr. Lollies,#3 of 25 Dessert in Amman \n#98 of 595 Places ...,5,11,"[(Mar-May, (3)), (Jun-Aug, (2)), (Sep-Nov, (5)...","[(Families, (1)), (Couples, (1)), (Solo, (0)),..."
3,"[English(45), Arabic(5), Spanish(3), ]",31.94913,35.9292,Mijana Restaurant,#41 of 545 Restaurants in Amman,4.5,56,"[(Mar-May, (12)), (Jun-Aug, (15)), (Sep-Nov, (...","[(Families, (5)), (Couples, (10)), (Solo, (1))..."
4,"[English(43), Italian(5), French(2), ]",31.95133,35.93271,Zorba Touristic Restaurant,#149 of 545 Restaurants in Amman,4,53,"[(Mar-May, (11)), (Jun-Aug, (16)), (Sep-Nov, (...","[(Families, (8)), (Couples, (4)), (Solo, (1)),..."
5,"[English(49), Portuguese(2), Arabic(1), ]",31.94932,35.924484,Caffe Strada,#35 of 545 Restaurants in Amman,4.5,53,"[(Mar-May, (9)), (Jun-Aug, (9)), (Sep-Nov, (11...","[(Families, (2)), (Couples, (9)), (Solo, (11))..."
6,"[English(74), Arabic(3), Spanish(2)]",31.950348,35.923878,Firefly Burger,#70 of 545 Restaurants in Amman,4,79,"[(Mar-May, (21)), (Jun-Aug, (18)), (Sep-Nov, (...","[(Families, (13)), (Couples, (10)), (Solo, (4)..."
7,"[English(60), Arabic(3), German(1), ]",31.973589,35.86889,Tche Tche,#376 of 545 Restaurants in Amman,3,67,"[(Mar-May, (15)), (Jun-Aug, (18)), (Sep-Nov, (...","[(Families, (11)), (Couples, (5)), (Solo, (5))..."
8,[English(3)],31.95063,35.92315,Mr. Krepe,#463 of 545 Restaurants in Amman,2.5,3,"[(Mar-May, (0)), (Jun-Aug, (0)), (Sep-Nov, (1)...","[(Families, (0)), (Couples, (1)), (Solo, (0)),..."
9,"[English(170), Italian(7), French(5), ]",31.94937,35.92642,Al-Quds,#12 of 545 Restaurants in Amman,4.5,197,"[(Mar-May, (47)), (Jun-Aug, (38)), (Sep-Nov, (...","[(Families, (30)), (Couples, (33)), (Solo, (22..."
10,"[English(53), Italian(3), Arabic(2), ]",31.95063,35.92315,Fatatri,#45 of 545 Restaurants in Amman,4.5,63,"[(Mar-May, (18)), (Jun-Aug, (8)), (Sep-Nov, (2...","[(Families, (12)), (Couples, (17)), (Solo, (2)..."


In [6]:
scraped_data2 = pd.DataFrame(scraped_data.dropna())

In [7]:
lat = [(float(x)) for x in scraped_data2['latitude']]
long = [(float(x)) for x in scraped_data2['longitude']]

In [9]:
import gmplot
# __, __ center map , __ zoom of map
gmap = gmplot.GoogleMapPlotter(31.950000, 35.920000, 14)
gmap.scatter(lat, long,'#3B0B39', size=40, marker=False)
#gmap.heatmap(lat,long)
gmap.draw("Amman_map.html")

In [10]:
#https://pypi.python.org/pypi/gmplot/1.0.5

In [11]:
#https://github.com/tcassou/gmaps    map examples

In [76]:
# --- languages dictionary to be converted into categorical variables (will continue it to do the variables later today)
from collections import Counter
import re
#def convert(string):
#    return Counter(re.findall('[A-Z][a-z]*', string))
import ast
languages_dictsList = []
for i in scraped_data2['languages']:
    lang = (re.findall('[A-Z][a-z]*', str(i)))
    freq = (re.findall('[1-9][0-9]*', str(i)))
    dictionary = dict(zip(lang, freq))
    languages_dictsList.append(dictionary)

In [77]:
print(languages_dictsList)

[{'English': '88', 'Italian': '3', 'German': '3'}, {'English': '10', 'French': '1'}, {'English': '45', 'Arabic': '5', 'Spanish': '3'}, {'English': '43', 'French': '2', 'Italian': '5'}, {'Portuguese': '2', 'English': '49', 'Arabic': '1'}, {'English': '74', 'Arabic': '3', 'Spanish': '2'}, {'English': '60', 'Arabic': '3', 'German': '1'}, {'English': '3'}, {'English': '170', 'French': '5', 'Italian': '7'}, {'English': '53', 'Arabic': '2', 'Italian': '3'}, {'English': '38', 'Arabic': '1', 'Dutch': '1'}, {'English': '5'}, {'English': '24', 'French': '1', 'Arabic': '2'}, {'English': '7'}, {'English': '24', 'Russian': '2', 'Arabic': '5'}, {'English': '14', 'Arabic': '1'}, {'English': '14', 'Arabic': '1'}, {'English': '2', 'Arabic': '1', 'Japanese': '1'}, {'Portuguese': '1', 'English': '65', 'Italian': '2'}, {'English': '2', 'Arabic': '1', 'Dutch': '1'}, {'English': '50', 'Spanish': '2', 'Italian': '2'}, {'English': '83', 'Arabic': '2', 'German': '3'}, {'English': '603', 'Italian': '33'}, {'Eng